# GRPO with Coding Task and Tools

## Import modules

In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

import asyncio
import time
import uuid
from dataclasses import dataclass
from typing import Any, Callable

import torch
import torch.nn.functional as F
import torchstore as ts
from datasets import load_dataset
from forge.actors._torchstore_utils import (
    get_dcp_whole_state_dict_key,
    get_param_prefix,
)
from forge.actors.generator import Generator as Policy
from forge.actors.reference_model import ReferenceModel
from forge.actors.replay_buffer import ReplayBuffer
from forge.actors.trainer import RLTrainer
from forge.cli.config import parse
from forge.controller.actor import ForgeActor
from forge.controller.provisioner import init_provisioner, shutdown
from forge.data.rewards import MathReward, ThinkingReward
from forge.observability.metric_actors import get_or_create_metric_logger
from forge.observability.metrics import record_metric, Reduce
from forge.observability.perf_tracker import Tracer

from forge.types import LauncherConfig, ProvisionerConfig
from forge.util.ops import compute_logprobs
from monarch.actor import endpoint
from omegaconf import DictConfig
from vllm.transformers_utils.tokenizer import get_tokenizer

import os
os.environ["MONARCH_HOSTMESH_V1"] = "1"
os.environ["TORCHSTORE_RDMA_ENABLED"] = "1"

/mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


INFO 10-16 20:07:55 [__init__.py:235] Automatically detected platform cuda.


## Define Data Structures

In [2]:
@dataclass
class Episode:
    # TODO: add adtional layer for multi-turn
    episode_id: str
    request: str
    policy_version: int
    pad_id: int
    request_len: int
    response_len: int
    target: Any | None = None
    # processed data
    response: str | None = None
    request_tokens: list[int] | None = None
    response_tokens: list[int] | None = None
    ref_logprobs: torch.Tensor | None = None
    reward: float | None = None
    advantage: float | None = None

    @property
    def request_tensor(self):
        tensor = torch.tensor(self.request_tokens, dtype=torch.long)
        if tensor.shape[0] < self.request_len:  # left pad
            diff = self.request_len - tensor.shape[0]
            tensor = F.pad(tensor, (diff, 0), value=self.pad_id)
        return tensor

    @property
    def response_tensor(self):
        tensor = torch.tensor(self.response_tokens, dtype=torch.long)
        if tensor.shape[0] < self.response_len:  # right pad
            diff = self.response_len - tensor.shape[0]
            tensor = F.pad(tensor, (0, diff), value=self.pad_id)
        return tensor


@dataclass
class Group:
    group_id: str
    episodes: list[Episode]

    @classmethod
    def new_group(
        cls,
        group_id: int,
        group_size: int,
        request: str,
        policy_version: int,
        pad_id: int,
        request_len: int,
        response_len: int,
        target: Any = None,
    ):
        episodes = []
        for _ in range(group_size):
            episodes.append(
                Episode(
                    episode_id=str(uuid.uuid4()),
                    request=request,
                    policy_version=policy_version,
                    pad_id=pad_id,
                    request_len=request_len,
                    response_len=response_len,
                    target=target,
                )
            )
        return cls(str(group_id), episodes)


def collate(batches: list[list[Episode]]):
    inputs = []
    targets = []
    for batch in batches:
        request = [e.request_tensor for e in batch]
        request = torch.stack(request)  # [b x s]

        response = [e.response_tensor for e in batch]
        response = torch.stack(response)  # [b x s]

        ref_logprobs = [e.ref_logprobs for e in batch]
        ref_logprobs = torch.stack(ref_logprobs).squeeze()  # [b x s]

        advantages = [e.advantage for e in batch]
        advantages = torch.tensor(advantages).unsqueeze(-1)  # [b x 1]

        pad_id = batch[0].pad_id
        mask = response != pad_id

        input = {"tokens": torch.cat([request, response], dim=1)}
        target = {
            "response": response,
            "ref_logprobs": ref_logprobs,
            "advantages": advantages,
            "padding_mask": mask,
        }
        inputs.append(input)
        targets.append(target)
    return inputs, targets

@dataclass
class DatasetActor(ForgeActor):
    """Actor wrapper for HuggingFace dataset to provide async interface."""

    path: str = "openai/gsm8k"
    revision: str = "main"
    data_split: str = "train"
    streaming: bool = True
    model: str = "Qwen/Qwen3-1.7B"

    @endpoint
    def setup(self):
        self._tokenizer = get_tokenizer(self.model)

        def gsm8k_transform(sample):
            system_prompt = """
            Put all your scratchpad work between <think> and </think> tags.
            Your final answer should be between <answer> and </answer> tags otherwise it will not be scored.
            """
            request: str = sample["question"]
            as_chat = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": request},
            ]
            formatted_request = self._tokenizer.apply_chat_template(
                as_chat,
                tokenize=False,
                add_generation_prompt=True,
            )
            target: str = sample["answer"]
            formatted_target = target.split("#### ")[1]
            return {"request": formatted_request, "target": formatted_target}

        ds = load_dataset(
            self.path, self.revision, split=self.data_split, streaming=self.streaming
        )
        ds = ds.map(gsm8k_transform)
        ds = ds.shuffle()
        self._iterator = iter(ds)

    @endpoint
    async def sample(self) -> dict[str, str] | None:
        try:
            sample = next(self._iterator)

            # Record dataset metrics
            record_metric("dataset/sample/count_samples_generated", 1, Reduce.SUM)
            record_metric(
                "dataset/sample/avg_sample_len",
                len(sample["request"]),
                Reduce.MEAN,
            )

            return sample
        except StopIteration:
            return None

    @endpoint
    async def pad_token(self):
        return self._tokenizer.pad_token_id

## Define loss

In [3]:
def simple_grpo_loss(
    logits: torch.Tensor,
    response: torch.Tensor,
    ref_logprobs: torch.Tensor,
    advantages: torch.Tensor,
    padding_mask: torch.Tensor,
    beta: float = 0.1,
) -> torch.Tensor:
    """
    Example GRPO Loss Function for RLTrainer
    """
    logprobs: torch.Tensor = compute_logprobs(logits, response)

    # Note: This is also available in losses.grpo_loss via `SimpleGRPOLoss`
    kl = torch.exp(ref_logprobs - logprobs) - (ref_logprobs - logprobs) - 1
    per_token_policy_loss = torch.exp(logprobs - logprobs.detach()) * advantages
    per_token_loss = -(per_token_policy_loss - beta * kl)
    loss = (
        ((per_token_loss * padding_mask).sum(dim=1))
        / (padding_mask.sum(dim=1).clamp(min=1.0))
    ).mean()
    return loss

## Define Reward

In [4]:
@dataclass
class RewardActor(ForgeActor):
    """Reward actor that uses a list of scoring functions."""

    reward_functions: list[Callable]

    @endpoint
    async def evaluate_response(self, prompt: str, response: str, target: str) -> float:
        total_rewards = 0.0
        for reward_fn in self.reward_functions:
            reward = reward_fn(prompt, response, target)
            total_rewards += reward

            # Get a name for the reward function (works for classes, functions, lambdas)
            reward_fn_name = getattr(
                reward_fn, "__name__", reward_fn.__class__.__name__
            )
            # per function reward
            record_metric(
                f"reward/evaluate_response/sum_{reward_fn_name}_reward",
                reward,
                Reduce.SUM,
            )
            record_metric(
                f"reward/evaluate_response/avg_{reward_fn_name}_reward",
                reward,
                Reduce.MEAN,
            )
            record_metric(
                f"reward/evaluate_response/std_{reward_fn_name}_reward",
                reward,
                Reduce.STD,
            )

            # avg total reward
            record_metric(
                "reward/evaluate_response/avg_total_reward",
                reward,
                Reduce.MEAN,
            )

            # count fn calls
            record_metric(
                f"reward/evaluate_response/count_{reward_fn_name}_calls",
                1,
                Reduce.SUM,
            )

        avg_reward = total_rewards / len(self.reward_functions)
        return avg_reward


@dataclass
class ComputeAdvantages(ForgeActor):
    """Compute advantages for GRPO using reward signals."""

    @endpoint
    async def compute(self, group: Group) -> list[float]:
        # TODO: add batch processing
        rewards = torch.tensor([[e.reward for e in group.episodes]])
        mean = rewards.mean(1, keepdim=True)
        std = rewards.std(1, keepdim=True)
        advantages = (rewards - mean) / (std + 1e-4)
        return advantages.squeeze(0).tolist()

In [5]:
async def drop_weights(version: int):
    print(f"Dropping weights @ version {version}")
    start_time = time.perf_counter()
    prefix = get_param_prefix(version)
    matching_keys = await ts.keys(prefix)
    # TODO: once we have something like `get_meta()` in torchstore, we can just
    # query the type of the object instead of relying on keys.
    dcp_key = get_dcp_whole_state_dict_key(version)
    if dcp_key in matching_keys:
        dcp_handle = await ts.get(dcp_key)
        dcp_handle.drop()
    for key in matching_keys:
        await ts.delete(key)
    elapsed = time.perf_counter() - start_time
    print(f"Dropped weights @ version {version}, took {elapsed:.2f} seconds")

## !!!!!!! Custom Testing !!!!!!!

In [6]:
#from actors.coder import SandboxedPythonCoder

## Setup Services

In [10]:
await trainer.push_weights.call(1)

[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/torch/cuda/memory.py:491: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
[0]   warnings.warn(


[0] [RLTrainer-0/1] 2025-10-16 20:11:48 INFO Pushing weights for policy version 1
[0] [RLTrainer-0/1] 2025-10-16 20:11:52 INFO Completed weights push in 3.96 seconds


ValueMesh({procs: 1}):
  (({'procs': 0/1}, None),)

In [13]:
get_param_prefix(1)

'policy_ver_0000000001'

In [14]:
keys = await ts.keys(get_param_prefix(1))
set(k.split(".")[0] for k in keys)

{'policy_ver_0000000001'}

In [7]:
from omegaconf import OmegaConf
from forge.cli.config import resolve_hf_hub_paths

cfg = OmegaConf.load('apps/grpo/qwen3_1_7b.yaml')
cfg = resolve_hf_hub_paths(cfg)
OmegaConf.resolve(cfg)

group_size = cfg.group_size # 8
max_req_tokens = cfg.max_req_tokens # 512
max_res_tokens = cfg.max_res_tokens # 512

metric_logging_cfg = cfg.get("metric_logging", {"console": {"log_per_rank": False}})
mlogger = await get_or_create_metric_logger()
await mlogger.init_backends.call_one(metric_logging_cfg)
await ts.initialize(strategy=ts.ControllerStorageVolumes())

dataloader, policy, trainer, replay_buffer, compute_advantages, ref_model, reward_actor = await asyncio.gather(
    DatasetActor.options(**cfg.actors.dataset).as_actor(**cfg.dataset),
    Policy.options(**cfg.services.policy).as_service(**cfg.policy),
    RLTrainer.options(**cfg.actors.trainer).as_actor(
        **cfg.trainer, loss=simple_grpo_loss
    ),
    ReplayBuffer.options(**cfg.actors.replay_buffer).as_actor(
        **cfg.replay_buffer, collate=collate
    ),
    ComputeAdvantages.options(**cfg.actors.compute_advantages).as_actor(),
    ReferenceModel.options(**cfg.services.ref_model).as_service(**cfg.ref_model),
    RewardActor.options(**cfg.services.reward_actor).as_service(
        reward_functions=[MathReward(), ThinkingReward()]
    ),
)

/mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happ

wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Spawning actor DatasetActor
Launcher not provided, remote allocations will not work.
Spawning service Generator
Spawning actor RLTrainer
Spawning actor ReplayBuffer
Spawning actor ComputeAdvantages
Spawning service ReferenceModel
Spawning service RewardActor


[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
[0]   warnings.warn(
[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
[0]   warnings.warn(
[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
[0]   warnings.warn(
[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
[0]   warnings.warn(
[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/si

[0] [RLTrainer-0/1] 2025-10-16 20:08:12 INFO Compiling loss
[0] INFO 10-16 20:08:14 [__init__.py:235] Automatically detected platform cuda.


...

[0] [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[0] [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[0] [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[0] [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[0] [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[0] INFO 10-16 20:08:35 [parallel_state.py:1102] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
[0] WARNING 10-16 20:08:35 [topk_topp_sampler.py:59] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
[0] INFO 10-16 20:08:35 [gpu_model_runner.py:1843] Starting to load model Qwen/Qwen3-1.7B...
[0] INFO 10-16 20:08:36 [gpu_model_runner.py:1875] Loading model from scratch...
[0] INFO 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.95it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  3.89it/s]
[0] 


[0] INFO 10-16 20:08:37 [default_loader.py:262] Loading weights took 0.63 seconds
[0] INFO 10-16 20:08:37 [gpu_model_runner.py:1892] Model loading took 3.2152 GiB and 1.116859 seconds
[0] INFO 10-16 20:08:45 [backends.py:530] Using cache directory: /mnt/home/pbontrager/.cache/vllm/torch_compile_cache/7ab64a271d/rank_0_0/backbone for vLLM's torch.compile
[0] INFO 10-16 20:08:45 [backends.py:541] Dynamo bytecode transform time: 7.57 s
[0] INFO 10-16 20:08:50 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 4.502 s
[0] INFO 10-16 20:09:03 [monitor.py:34] torch.compile takes 7.57 s in total
[0] INFO 10-16 20:09:04 [gpu_worker.py:255] Available KV cache memory: 62.38 GiB
[0] INFO 10-16 20:09:04 [kv_cache_utils.py:833] GPU KV cache size: 584,000 tokens
[0] INFO 10-16 20:09:04 [kv_cache_utils.py:837] Maximum concurrency for 40,960 tokens per request: 14.26x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:01<00:00, 38.63it/s]


[0] INFO 10-16 20:09:06 [gpu_model_runner.py:2485] Graph capturing finished in 2 secs, took 0.60 GiB


## Rollout Loop

In [8]:
async def continuous_rollouts():
    rollout_count = 0
    pad_id = await dataloader.pad_token.call_one()
    while True:
        t = Tracer("main_perf/continuous_rollouts")
        t.start()
        sample = await dataloader.sample.call_one()
        if sample is None:
            print("Dataloader is empty, exiting continuous rollout")
            return

        t.step("data_loading")

        prompt, target = sample["request"], sample["target"]
        responses = await policy.generate.route(prompt)
        # TODO: this shall be part of the responses metadata instead of a separate call
        version = await policy.get_version.route()

        t.step("policy_generation")

        assert (
            len(responses) > 0
        ), "Sanity check: Responses should NEVER return empty"
        assert (
            version := responses[0].generator_version
        ) is not None, "Response must indicate a version"
        group = Group.new_group(
            group_id=rollout_count,
            group_size=group_size,
            request=prompt,
            policy_version=version,
            pad_id=pad_id,
            request_len=max_req_tokens,
            response_len=max_res_tokens,
            target=target,
        )

        input_ids = torch.ones(
            (group_size, max_req_tokens + max_res_tokens),
            dtype=torch.long,
            device="cuda",
        )
        # Populate episode info and calculate rewards
        for i, (episode, response) in enumerate(zip(group.episodes, responses)):
            episode.request_tokens = response.prompt_ids
            episode.response_tokens = response.token_ids
            episode.response = response.text
            input_ids[i, :max_req_tokens] = episode.request_tensor
            input_ids[i, max_req_tokens:] = episode.response_tensor
            episode.reward = await reward_actor.evaluate_response.route(
                prompt=prompt, response=response.text, target=target
            )

        t.step("reward_evaluation")

        ref_logprobs = await ref_model.forward.route(
            input_ids, max_req_tokens, return_logprobs=True
        )
        t.step("reference_model_calculate_logprobs")

        for i, episode in enumerate(group.episodes):
            episode.ref_logprobs = ref_logprobs[i]
        del ref_logprobs, input_ids
        t.step("compute_logprobs")

        # Calculate advantages and add to replay buffer
        advantages = await compute_advantages.compute.call_one(group)
        for episode, advantage in zip(group.episodes, advantages):
            episode.advantage = advantage
            await replay_buffer.add.call_one(episode)

        # Log metrics
        rollout_count += 1
        record_metric(
            "main/continuous_rollouts/count_rollout_iterations", 1, Reduce.SUM
        )
        t.stop()

## Training Loop

In [9]:
async def continuous_training():
    training_step = 0
    restart_tracer = True  # Flag to control when to restart tracer
    # update_task = asyncio.sleep(0) philip
    while True:
        # Restart tracer when needed (initial start or after completing a training step)
        # Otherwise, we cannot measure time waiting for buffer
        if restart_tracer:
            t = Tracer("main_perf/continuous_training")
            t.start()
            restart_tracer = False

        batch = await replay_buffer.sample.call_one(
            curr_policy_version=training_step
        )
        if batch is None:
            await asyncio.sleep(0.1)
        else:
            t.step("waiting_for_buffer")

            inputs, targets = batch
            await trainer.train_step.call(inputs, targets)
            training_step += 1
            t.step("train_step")

            await trainer.push_weights.call(training_step)
            t.step("push_weights")

            await policy.update_weights.fanout(training_step)
            #await update_task philip
            update_task = asyncio.create_task(policy.update_weights.fanout(training_step))
            t.step("update_weights")

            if training_step >= 2:
                await drop_weights(training_step - 1)
                #asyncio.create_task(drop_weights(training_step - 1)) philip
                t.step("drop_weights")

            t.stop()
            restart_tracer = True

            # Flush metrics every training step to WandB
            await mlogger.flush.call_one(training_step)

## Run

In [ ]:
num_rollout_threads = 1
num_training_threads = 1

rollout_tasks = [
    asyncio.create_task(continuous_rollouts()) for _ in range(num_rollout_threads)
]
training_task = asyncio.create_task(continuous_training())

try:
    await asyncio.gather(*rollout_tasks, training_task)
except KeyboardInterrupt:
    print("Training interrupted by user")
    for rollout_task in rollout_tasks:
        rollout_task.cancel()
    training_task.cancel()

/tmp/ipykernel_1858026/82490467.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(self.request_tokens, dtype=torch.long)
/tmp/ipykernel_1858026/82490467.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(self.response_tokens, dtype=torch.long)
[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/torch/cuda/memory.py:491: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
[0]   warnings.warn(
[0] /tmp/ipykernel_1858026/82490467.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or 

[0] [ReferenceModel-0/1] 2025-10-16 19:37:27 INFO [GC] Performing periodic GC collection took 0.00 seconds


[0] /mnt/home/pbontrager/.conda/envs/forge/lib/python3.10/site-packages/torch/cuda/memory.py:491: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
[0]   warnings.warn(


[0] [RLTrainer-0/1] 2025-10-16 19:37:27 INFO Pushing weights for policy version 1
[0] [ReferenceModel-0/1] 2025-10-16 19:37:28 INFO [GC] Performing periodic GC collection took 0.00 seconds
[0] [ReferenceModel-0/1] 2025-10-16 19:37:30 INFO [GC] Performing periodic GC collection took 0.00 seconds
[0] [RLTrainer-0/1] 2025-10-16 19:37:31 INFO Completed weights push in 4.24 seconds
WandbBackend: Logged 91 metrics at global_step 1
=== [global_logger_2nwJ_r0] - METRICS STEP 1 ===
  buffer/add/count_episodes_added: 40.0
  buffer/evict/avg_policy_age: 0.0
  buffer/evict/max_policy_age: 0.0
  buffer/evict/sum_episodes_evicted: 0.0
  buffer/sample/avg_data_utilization: 1.9565217391304348
  buffer/sample/count_sample_requests: 59.0
  buffer_perf/sample/total_duration_avg_s: 8.071606221087909e-05
  buffer_perf/sample/total_duration_max_s: 0.002548671793192625
  dataset/sample/avg_sample_len: 442.5
  dataset/sample/count_samples_generated: 6.0
  generator/generate/avg_tokens_generated: 447.675
  gen

[0] /tmp/ipykernel_1858026/82490467.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
[0] /tmp/ipykernel_1858026/82490467.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

[0] [RLTrainer-0/1] 2025-10-16 19:50:44 INFO Pushing weights for policy version 108
[0] [RLTrainer-0/1] 2025-10-16 19:50:46 INFO Completed weights push in 2.57 seconds
[0] INFO 10-16 19:50:47 [block_pool.py:321] Successfully reset prefix cache
[0] [Generator-0/1] 2025-10-16 19:50:47 INFO Weight update completed (now v107)
Dropping weights @ version 107
WandbBackend: Logged 96 metrics at global_step 108
=== [global_logger_2nwJ_r0] - METRICS STEP 108 ===
  buffer/add/count_episodes_added: 8.0
  buffer/evict/avg_policy_age: 1.0
  buffer/evict/max_policy_age: 1.0
  buffer/evict/sum_episodes_evicted: 16.0
  buffer/sample/avg_data_utilization: 1.9166666666666667
  buffer/sample/count_sample_requests: 12.0
  buffer_perf/sample/total_duration_avg_s: 0.00012942620863517126
  buffer_perf/sample/total_duration_max_s: 0.000685298815369606
  dataset/sample/avg_sample_len: 461.0
  dataset/sample/count_samples_generated: 1.0
  generator/generate/avg_tokens_generated: 512.0
  generator/generate/count_

[0] /tmp/ipykernel_1858026/82490467.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
[0] /tmp/ipykernel_1858026/82490467.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


[0] [RLTrainer-0/1] 2025-10-16 19:50:55 INFO Pushing weights for policy version 109
[0] [ReferenceModel-0/1] 2025-10-16 19:50:56 INFO [GC] Performing periodic GC collection took 0.00 seconds
[0] [RLTrainer-0/1] 2025-10-16 19:50:58 INFO Completed weights push in 3.41 seconds
Dropping weights @ version 108
WandbBackend: Logged 96 metrics at global_step 109
=== [global_logger_2nwJ_r0] - METRICS STEP 109 ===
  buffer/add/count_episodes_added: 24.0
  buffer/evict/avg_policy_age: 0.9905660377358491
  buffer/evict/max_policy_age: 1.0
  buffer/evict/sum_episodes_evicted: 16.0
  buffer/sample/avg_data_utilization: 1.9811320754716981
  buffer/sample/count_sample_requests: 74.0
  buffer_perf/sample/total_duration_avg_s: 8.4217033676199e-05
  buffer_perf/sample/total_duration_max_s: 0.0006482847966253757
  dataset/sample/avg_sample_len: 573.0
  dataset/sample/count_samples_generated: 3.0
  generator/generate/avg_tokens_generated: 512.0
  generator/generate/count_requests: 3.0
  generator/generate/

[0] /tmp/ipykernel_1858026/82490467.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
[0] /tmp/ipykernel_1858026/82490467.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


[0] [RLTrainer-0/1] 2025-10-16 19:50:59 INFO Pushing weights for policy version 110


## Shutdown

In [ ]:
await mlogger.shutdown.call_one()
await asyncio.sleep(2)

await asyncio.gather(
    DatasetActor.shutdown(dataloader),
    policy.shutdown(),
    RLTrainer.shutdown(trainer),
    ReplayBuffer.shutdown(replay_buffer),
    ComputeAdvantages.shutdown(compute_advantages),
    ref_model.shutdown(),
    reward_actor.shutdown(),
)
await shutdown()